In [17]:
import multiprocessing
import re
from io import StringIO

import pandas as pd
import numpy as np

from emukit.core import ParameterSpace, ContinuousParameter
from emukit.core.initial_designs import RandomDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

In [18]:
target_name = 'SEIZ_updated'
# n_runs = 5
# max_parallel_runs = 8
# threads_per_run = multiprocessing.cpu_count()//min(n_runs,max_parallel_runs)
threads_per_run = multiprocessing.cpu_count()

parameter_space = ParameterSpace([ContinuousParameter('p', 0, 1),
                                  ContinuousParameter('l', 0, 1),
                                  ContinuousParameter('epsilon', 2, 2)])

In [19]:
def run_job(p, l, epsilon, clear_results=False):
    with open(f'{target_name}_editable.fred', 'r') as file:
        filedata = file.read()

    for target, value in {'{p}': p, '{l}': l, '{epsilon}': epsilon}.items():
        filedata = filedata.replace(target, str(value))

    with open(f'{target_name}_out.fred', 'w') as file:
        file.write(filedata)
        
    if clear_results:
        !yes | fred_clear_all_results
        
    # !fred_job -p {target_name}_out.fred -k {target_name}_{p}_{l}_{epsilon}_run -n {n_runs} -m {max_parallel_runs} -t {threads_per_run}
    !fred_job -p {target_name}_out.fred -k {target_name}_{p}_{l}_{epsilon}_run -t {threads_per_run}
    !fred_csv -k {target_name}_{p}_{l}_{epsilon}_run > results/{target_name}_{p}_{l}_{epsilon}_run.csv

    # with open(f'results/{target_name}_{p}_{l}_{epsilon}_run.csv', 'r') as csv:
    #     results = csv.read()
    # 
    # results = re.split(' \nRun,\d\n', results)[1:]
    # return [pd.read_csv(StringIO(data)) for data in results]
    return pd.read_csv(f'results/{target_name}_{p}_{l}_{epsilon}_run.csv', header=1)

In [20]:
def sample(X, **kwargs):
    X = np.array(X)
    
    if X.ndim < 2:
        X = np.expand_dims(X, 0)
    elif X.ndim > 2:
        raise Exception(f'X has too many dimensions (ndim={X.ndim}, must be 1 or 2)')
    if X.shape[-1] != 3:
        raise Exception(f'X has the wrong number of variables (variables={X.shape[-1]}, must be 3)')
    
    Y = []

    for (p, l, epsilon) in X: # TODO: parallelize this?
        data = run_job(p, l, epsilon, **kwargs)
        # Y.append([run['ADOPT.I'].iat[-1] for run in data])
        Y.append([data['ADOPT.I'].iat[-1]])
        
    return np.vstack(Y)

In [21]:
design = RandomDesign(parameter_space)
num_data_points = 5
X = design.get_samples(num_data_points)

In [22]:
X

array([[0.78163638, 0.31046252, 2.        ],
       [0.09325979, 0.62248035, 2.        ],
       [0.17222669, 0.33790588, 2.        ],
       [0.41210966, 0.60329941, 2.        ],
       [0.03328561, 0.55815782, 2.        ]])

In [23]:
Y = sample(X, clear_results=True)

You are about to delete /home/nyoma/FRED/RESULTS. This cannot be undone.
Proceed? yes/no [no]
/home/nyoma/FRED/RESULTS deleted
yes: standard output: Broken pipe
fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_0.7816363781058014_0.3104625228956649_2.0_run 1
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/1/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/1/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/1/OUT/RUN1/LOG)
You are about to delete /home/nyoma/FRED/RESULTS. This cannot be undone.
Proceed? yes/no [no]
/home/nyoma/FRED/RESULTS deleted
yes: standard output: Broken pipe
fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) 

In [24]:
Y

array([[44633],
       [30563],
       [43536],
       [33831],
       [34532]])

In [25]:
model_gpy = GPRegression(X,Y)
model_emukit = GPyModelWrapper(model_gpy)

In [26]:
expected_improvement = ExpectedImprovement(model=model_emukit)

In [27]:
bayesopt_loop = BayesianOptimizationLoop(model=model_emukit,
                                         space=parameter_space,
                                         acquisition=expected_improvement,
                                         batch_size=1)

In [28]:
max_iterations = 30
bayesopt_loop.run_loop(sample, max_iterations)

fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_0.9774898542528399_0.9761446820634082_2.0_run 2
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/2/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/2/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/2/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_0.9749203925289539_0.9926093282313667_2.0_run 3
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/3/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/3/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/3/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_0.97096798709076_0.976086638261783_2.0_run 4
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/4/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/4/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/4/OUT/RUN1/LOG)
fred_compile SEIZ_updated_out.fred ...


...

 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_0.9562060536682474_0.983021626491473_2.0_run 30
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/30/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/30/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/30/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


fred_compile SEIZ_updated_out.fred ...
No errors found.

FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_updated_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_updated_0.9865867589672788_0.9859985310129568_2.0_run 31
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/31/WORK/SEIZ_updated_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/31/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/31/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


In [29]:
results = bayesopt_loop.get_results()

In [30]:
print(results.minimum_location, results.minimum_value)
print(results.best_found_value_per_iteration)

[0.96669887 0.99949393 2.        ] 104.0
[44633. 30563. 30563. 30563. 30563.  2824.   952.   952.   554.   554.
   554.   554.   554.   554.   554.   379.   379.   106.   106.   106.
   106.   106.   106.   106.   106.   106.   106.   104.   104.   104.
   104.   104.   104.   104.   104.]
